In [6]:
import requests
from bs4 import BeautifulSoup as bsp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

import json
import time
from datetime import datetime, timedelta
from selenium import webdriver

In [138]:
# PYTHON으로 SQLITE3의 DB에 데이터 담기
dbname='wadiz.db'
def insData(d, tblname='wadiztable'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select nkey from '+tblname+' where nkey="'+d[-1]+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            print('-',end='')
        
        else:
            insql='insert into '+tblname+' (ntitle,ncontent,npercent,namount,nsup,nkey) values (?,?,?,?,?,?)'
            cur.execute(insql,d)
            print('.',end='')

In [2]:
# 무한 스크롤 코드
driver = webdriver.Chrome('C:\work37\hackerton\chromedriver.exe')
driver.get("https://www.wadiz.kr/web/wreward/category/288?keyword=&endYn=ALL&order=recommend") 

elements = driver.find_element_by_xpath(f'//*[@id="main-app"]/div[2]/div/div[5]/div[2]/div[2]/div/button')
time.sleep(0.5)
driver.execute_script("arguments[0].click();", elements)
time.sleep(0.5)

SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

In [23]:
# 무한 스크롤 된 상태에서 주소 담는 코드
hrefs=[]
tables =driver.find_element_by_class_name('CommonCard_container__e_ebQ') 
table =tables.find_element_by_class_name('CardLink_link__1k83H')
elems = table.find_elements_by_xpath("//a[@href]")
for elem in elems:
    hrefs.append(elem.get_attribute("href"))

In [4]:
hrefs=[]
tables =driver.find_elements_by_class_name("CommonCard_image__vaqkf")
for table in tables:
    hrefs.append(table.get_attribute('href'))
len(hrefs)

In [139]:
for i in range(200):
    url = hrefs[i]
    reqheader = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} 
    src=requests.get(url,headers=reqheader)
    cont=src.content.strip() 
    html=bsp(cont,'html.parser')

    title=html.select('h2.title a')
    ntitle=title[0].text

    contents=html.select('.inner-contents p')
    cont=[]
    for content in contents:
            cont.append(content.text)
    ncontent=' '.join(cont)

    percent=html.select('p.achievement-rate strong')
    npercent=percent[0].text

    amount=html.select('p.total-amount strong')
    namount=amount[0].text

    support=html.select('p.total-supporter strong')
    nsup=support[0].text

    nkey= url[-6:]

    d=(ntitle, ncontent, npercent, namount, nsup, nkey)
    insData(d)

........................................................................................................................................................................................................